Experiment 2: Vary the momentum value
    where m = 0,0.25,5

In [1]:
#Import the necessary libraries
import csv
import time
import matplotlib
import numpy
from scipy.special import expit

In [2]:
#Initializing the values 
epochs = 50
eta = 0.1
#To vary the momentum vale, change the values of m - 0.25,0.5
m = 0

In [3]:
#bias input
bias_input = 1

In [4]:
#input layer weight matrix.
weight_input = numpy.random.uniform(-0.05,0.05,(784,100))

#hidden layer weight matrix.
weight_hidden = numpy.random.uniform(-0.05,0.05,(100,10))

In [5]:
#bias weights for hidden layer.
weight_ib = numpy.random.uniform(-0.05,0.05,(1,100))

#bias weights for output layer.
weight_ob = numpy.random.uniform(-0.05,0.05,(1,10))

In [6]:
#matrix to store previous hidden layer bias weights
it_weight_ib = numpy.zeros((1,100))

#matrix to store previous output layer bias weights.
it_weight_ob = numpy.zeros((1,10))

#matrix to store previous input-hidden layer weights.
it_weight_ih = numpy.zeros((784,100))

#matrix to store previous hidden-output layer weights.
it_weight_ho = numpy.zeros((100,10))

In [ ]:
inputs = numpy.genfromtxt('mnist_train.csv',delimiter=',')
test = numpy.genfromtxt('mnist_test.csv',delimiter=',') 
#Each output unit corresponds to one of the ten classes ( 0 to 9)
classes = numpy.array([0,1,2,3,4,5,6,7,8,9])

In [7]:
def sigmoid(n):
    return expit(n)

In [ ]:
def feedForward(e,inputs,labels,setName):
    global weight_input,weight_hidden,bias_input,weight_ib,weight_ob,it_weight_ib,it_weight_ob,it_weight_ih,it_weight_ho
     actual = []
    predictions = []
        
    for i in range(inputs.shape[0]):
        actual.append(labels[i])
        inputData = numpy.reshape(inputs[i],(1,inputs[i].shape[0])) l1 = bias_input*w1b
        z1 = numpy.dot(inputData,w1) + l1
        a1 = sigmoid(z1)
        l2 = bias_input*weight_ob
        z2 = numpy.dot(a1,weight_hidden) + l2
        a2 = sigmoid(z2)
        prediction = numpy.argmax(a2)
        predictions.append(prediction)
        
        if(setName == 'training' and e != 0):
             it_weight_ib,it_weight_ob,it_weight_ih,it_weight_ho = learn(e,inputData,a1,a2,prediction,labels[i])
                
    generateConfusionMatrix(e,classes,numpy.array(actual),numpy.array(predictions),setName,it_weight_ib.shape[1])
    time.sleep(2)

In [ ]:
def learn(epochs,inputs,a1,a2,prediction,label):
    global bias_input,weight_hidden,weight_ob,eta,m,it_weight_ob,it_weight_ho
    hotVector = numpy.insert((numpy.zeros((1,9))+0.1),label,0.9)
    errors2 = a2 * (1 - a2) * (hotVector - a2)
    errors1 = a1 * (1 - a1) * numpy.dot(errors2,numpy.transpose(w2))
    oldweight_hidden = updateWeightsH(epochs,a1,errors2)
    oldweight_input = updateWeightsI(epochs,inputs,errors1)
    return oldweight_input[0],oldweight_input[1],oldweight_hidden[0],oldweight_hidden[1]

In [ ]:
def updateWeightsH(epochs,a1,errors):
    global bias_input,weight_ob,weight_hidden,eta,m,it_weight_ho,it_weight_ob 
    deltaw2b = (eta * numpy.dot(errors,biasInput)) + (m * it_weight_ob )
    weight_ob = weight_ob + deltaw2b
    deltaw2 = (eta * numpy.dot(numpy.transpose(a1),errors)) + (m * it_weight_ho) 
    weight_hidden = weight_hidden + deltaw2
    return deltaw2b,deltaw2

In [ ]:
#update
def updateWeightsI(epochs,inputs,errors):
    global bias_input,weight_ib,weight_input,eta,m,it_weight_ib,it_weight_ih
    deltaw1b = (eta * numpy.dot(errors,bias_input)) + (m * it_weight_ib)
    weight_ib = weight_ib + deltaw1b
    deltaw1 = (eta * numpy.dot(numpy.transpose(inputs),errors)) + (m * it_weight_ih)
    weight_input = weight_input + deltaw1
    return deltaw1b,deltaw1

In [ ]:
def grabLabels(inputs):
        labels = numpy.zeros(inputs.shape[0],numpy.int32) 
        for i in range(inputs.shape[0]):
             labels[i] = inputs[i,0]
        return labels

In [ ]:
def preProcessInput(inputs):
    for i in range(inputs.shape[0]):
        inputs[i,:] = inputs[i,:]/255.0
        return inputs[:,1:]

In [ ]:
def generateConfusionMatrix(epoch,classes,actual,predictions,setName,n): 
    global eta
    confusionMatrix = numpy.zeros((len(classes),len(classes)),numpy.int32) 
    for a,p in zip(actual,predictions):
         confusionMatrix[a][p] = confusionMatrix[a][p] + 1
     #end
    printConfusionMatrix(confusionMatrix)
    accuracy = (actual == predictions).sum()/float(len(actual))*100 
    print(setName+" - accuracy:",accuracy)
    writeData(eta,epoch,accuracy,setName,n)

In [ ]:
def writeData(eta,epoch,accuracy,setName,n):
    global alpha,inputs
    f = open('exp-2 0-'+setName+'-.csv', 'a',newline='') 
    with f:
        writer = csv.writer(f)
        writer.writerow([epoch,accuracy])

In [ ]:
def printConfusionMatrix(confusionMatrix): 
    print("confusion matrix\n")
    for i in range(confusionMatrix.shape[0]):
        print(confusionMatrix[i])

In [ ]:
def start():
       global epochs,inputs,test,testData,it_weight_ib,it_weight_ob,it_weight_ih,it_weight_ho
        labels = grabLabels(inputs)
        inputs = preProcessInput(inputs)
        testLabels = grabLabels(test)
        testData = preProcessInput(test)
        for e in range(epochs):
             print("\nepoch : ",e)
            feedForward(e,inputs,labels,'training') 
            feedForward(e,testData,testLabels,'testing')

In [ ]:
numpy.random.seed(None)
start()

In [ ]:
# Plotting the graph
import matplotlib.pyplot as plt 
import numpy as np
x1, y1 = np.loadtxt('exp-2 0training.csv', delimiter=',', unpack=True) 
x2, y2 = np.loadtxt('exp-2 0testing.csv', delimiter=',', unpack=True) 
fig, ax = plt.subplots()
plt.plot(x1,y1, label='Training Set')
plt.plot(x2,y2, label='Testing Set')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend(loc='lower right')
plt.show()